In [1]:
#Importing datasets

import pandas as pd
accidents_2017 = pd.read_csv("../datasets/1.-Transportation/accidents-2017.csv")
motos_2017 = pd.read_csv("../datasets/1.-Transportation/2017_cilindrada_motos.csv")


accidents_2016 = pd.read_csv("../datasets/1.-Transportation/2016_accidents_gu_bcn.csv")
motos_2016 = pd.read_csv("../datasets/1.-Transportation/2016_cilindrada_motos (1).csv")

In [2]:
#Creating  "One-Hot Coding" column to help calculating bulk number of accidents when the dataframe is grouped,
#Also adding a column for year

#accidents.insert(loc= len(accidents.columns),column="Accidents", value = 1)


accidents_2016.insert(loc= len(accidents_2016.columns),column="Accidents", value = 1)
accidents_2017.insert(loc= len(accidents_2017.columns),column="Accidents", value = 1)
#accidents_2016.insert(loc= len(accidents_2016.columns),column="Year", value = 2016)
#accidents_2017.insert(loc= len(accidents_2017.columns),column="Year", value = 2017)



#Checking for empty values that might need cleaning (Spoiler:Everything is fine)

print(accidents_2016.info())
print(motos_2016.info())
print(accidents_2017.info())
print(motos_2017.info())

#Reducing datasets only to the columns that will be used during the analysis, and renaming when necessary(CAT>ENG)

accidents_2017 = accidents_2017[["District Name","Mild injuries","Serious injuries","Victims","Accidents"]]
accidents_2016 = accidents_2016[["Nom_districte","Numero_lesionats_lleus","Numero_lesionats_greus","Numero_victimes","Numero_morts","Accidents"]]

accidents_2016.rename(columns={"Nom_districte":"District Name",
"Numero_lesionats_lleus":"Mild injuries","Numero_lesionats_greus":"Serious injuries"
,"Numero_victimes":"Victims","Numero_morts":"Deaths"}, inplace=True)

accidents_2016.replace(to_replace = "Desconegut", value ="Unknown",inplace=True)
accidents_2016 = accidents_2016.groupby(["District Name"]).sum()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10140 entries, 0 to 10139
Data columns (total 28 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Numero_expedient           10140 non-null  object 
 1   Codi_districte             10140 non-null  int64  
 2   Nom_districte              10140 non-null  object 
 3   Codi_barri                 10140 non-null  int64  
 4   Nom_barri                  10140 non-null  object 
 5   Codi_carrer                10140 non-null  int64  
 6   Nom_carrer                 10140 non-null  object 
 7   Num_postal                 10135 non-null  object 
 8   Descripcio_dia_setmana     10140 non-null  object 
 9   Dia_setmana                10140 non-null  object 
 10  Descripcio_tipus_dia       10140 non-null  object 
 11  Any                        10140 non-null  int64  
 12  Mes_any                    10140 non-null  int64  
 13  Nom_mes                    10140 non-null  obj

In [4]:
#Changing names from catalan to english, also aggregating the bike dataset by pivot table 

motos_2017 = motos_2017.replace(to_replace = "No consta", value ="Unknown")
motos_2017 = motos_2017.replace(to_replace = {"126 a 200":"126 a 200 cc", "Fins a 75 cc":"35 a 75 cc","Més de 750 cc":"750 a 1200 cc" })
motos_2016 = motos_2016.replace(to_replace = "No consta", value ="Unknown")
motos_2016 = motos_2016.replace(to_replace = {"126 a 200":"126 a 200 cc", "Fins a 75 cc":"35 a 75 cc","Més de 750 cc":"750 a 1200 cc" })

motos_distr = pd.pivot_table(data=motos_2016 ,values="Nombre_motos", index = "Nom_Districte",
                             columns="Cilindrada_motos",margins=True,aggfunc=sum)

accidents_2016

,Mild injuries,Serious injuries,Victims,Deaths,Accidents
District Name,,,,,
Ciutat Vella,538,15,554,1,531
Eixample,3712,50,3767,5,3118
Gràcia,529,6,536,1,501
Horta-Guinardó,876,7,885,2,727
Les Corts,879,21,904,4,739
Nou Barris,602,18,623,3,515
Sant Andreu,701,11,713,1,560
Sant Martí,1511,19,1530,0,1260
Sants-Montjuïc,1315,27,1347,5,1106


In [5]:
#Merging both datasets

df_merged=accidents_2016.merge(right=motos_distr, how="left",left_index=True,right_index=True)
df_merged=df_merged.rename({'All': 'total_bikes'}, axis=1)
df_merged=df_merged.sort_values(by="Accidents", ascending=False)

lst = ['126 a 200 cc', '201 a 250 cc', '251 a 300 cc', '301 a 500 cc',
       '35 a 75 cc', '501 a 750 cc', '75 a 125 cc', '750 a 1200 cc']
       
#for i in lst:
#df_merged[i]=df_merged[i]/df_merged['total_bikes']

In [8]:
#Correlacion
correlation = df_merged.corr()

In [13]:
correlation =correlation.sort_values(by="Deaths", ascending=False)
correlation

,Mild injuries,Serious injuries,Victims,Deaths,Accidents,126 a 200 cc,201 a 250 cc,251 a 300 cc,301 a 500 cc,35 a 75 cc,501 a 750 cc,75 a 125 cc,750 a 1200 cc,total_bikes
Deaths,0.524559,0.656754,0.528481,1.000000,0.529495,0.699141,0.667935,0.685298,0.689327,0.732484,0.531288,0.635194,0.589882,0.654947
35 a 75 cc,0.569445,0.498727,0.570028,0.732484,0.583833,0.963626,0.880812,0.946150,0.884295,1.000000,0.661140,0.869535,0.813243,0.885355
126 a 200 cc,0.711856,0.590670,0.711664,0.699141,0.721738,1.000000,0.968910,0.987042,0.967010,0.963626,0.823407,0.962973,0.924355,0.971564
301 a 500 cc,0.812165,0.706342,0.812147,0.689327,0.816617,0.967010,0.995475,0.964639,1.000000,0.884295,0.932079,0.992503,0.980052,0.995963
251 a 300 cc,0.659925,0.531930,0.659618,0.685298,0.665806,0.987042,0.964474,1.000000,0.964639,0.946150,0.835208,0.953774,0.910890,0.964865
201 a 250 cc,0.803715,0.686835,0.803501,0.667935,0.809669,0.968910,1.000000,0.964474,0.995475,0.880812,0.934781,0.997535,0.982683,0.999031
Serious injuries,0.922693,1.000000,0.925064,0.656754,0.924399,0.590670,0.686835,0.531930,0.706342,0.498727,0.719403,0.685210,0.758550,0.680884
total_bikes,0.801578,0.680884,0.801284,0.654947,0.808172,0.971564,0.999031,0.964865,0.995963,0.885355,0.931677,0.998939,0.983266,1.000000
75 a 125 cc,0.808761,0.685210,0.808384,0.635194,0.815823,0.962973,0.997535,0.953774,0.992503,0.869535,0.940215,1.000000,0.986322,0.998939
750 a 1200 cc,0.876207,0.758550,0.875802,0.589882,0.881035,0.924355,0.982683,0.910890,0.980052,0.813243,0.955551,0.986322,1.000000,0.983266
